# 날씨데이터 가공

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.rc('font', family='Malgun Gothic')

#경고 끄기
import warnings
warnings.filterwarnings(action='ignore')

## 탐색적 분석
- 일사 vs 일조

일사량과 일조시간 중 일조시간을 선택하기로 한다.<br>
일사량은 태양의 복사에너지가 지면에 닿는 양, 일조 시간은 햇빛이 비치는 시간<br>
합계 일사는 데이터가 거의 없음<br>

- 정선 없앰
  
  2008년 이전데이터 미확보

In [2]:
df = pd.read_csv('19802019.csv')

In [3]:
df.columns

Index(['지점', '일시', '평균기온(°C)', '최저기온(°C)', '최저기온 시각(hhmi)', '최고기온(°C)',
       '최고기온 시각(hhmi)', '강수 계속시간(hr)', '10분 최다 강수량(mm)', '10분 최다강수량 시각(hhmi)',
       '1시간 최다강수량(mm)', '1시간 최다 강수량 시각(hhmi)', '일강수량(mm)', '최대 순간 풍속(m/s)',
       '최대 순간 풍속 풍향(16방위)', '최대 순간풍속 시각(hhmi)', '최대 풍속(m/s)', '최대 풍속 풍향(16방위)',
       '최대 풍속 시각(hhmi)', '평균 풍속(m/s)', '풍정합(100m)', '최다풍향(16방위)',
       '평균 이슬점온도(°C)', '최소 상대습도(%)', '최소 상대습도 시각(hhmi)', '평균 상대습도(%)',
       '평균 증기압(hPa)', '평균 현지기압(hPa)', '최고 해면기압(hPa)', '최고 해면기압 시각(hhmi)',
       '최저 해면기압(hPa)', '최저 해면기압 시각(hhmi)', '평균 해면기압(hPa)', '가조시간(hr)',
       '합계 일조 시간(hr)', '1시간 최다일사 시각(hhmi)', '1시간 최다일사량(MJ/m2)', '합계 일사(MJ/m2)',
       '일 최심신적설(cm)', '일 최심신적설 시각(hhmi)', '일 최심적설(cm)', '일 최심적설 시각(hhmi)',
       '합계 3시간 신적설(cm)', '평균 전운량(1/10)', '평균 중하층운량(1/10)', '평균 지면온도(°C)',
       '최저 초상온도(°C)', '평균 5cm 지중온도(°C)', '평균 10cm 지중온도(°C)',
       '평균 20cm 지중온도(°C)', '평균 30cm 지중온도(°C)', '0.5m 지중온도(°C)',
       '1.0m 지중온도(°C)', '1.5m 지중온도(°C)', '3.0m 지중온도(°C)',

In [4]:
# 261-해남 216-태백 105-강릉
df = df[['지점','일시','평균기온(°C)','최고기온(°C)','최저기온(°C)','합계 일조 시간(hr)','1시간 최다강수량(mm)','일강수량(mm)','최대 풍속(m/s)']]
df['지점_nm'] = df['지점'].apply(lambda x : '해남' if x==261 else('태백' if x==216 else ('정선' if x==217 else '강릉')))
df['년월'] = df['일시'].apply(lambda x : x[:-3]) # yyyy-mm형식
df['월'] = df['년월'].apply(lambda x : x[5:]) # mm형식
df['도'] = df['지점'].apply(lambda x : '전라도' if x==261 else '강원도') # 강원도 전라도로 변환

In [5]:
# 결측값 %확인
print(df.isna().sum())
print('='*100)
print(df.shape)

지점                   0
일시                   0
평균기온(°C)            41
최고기온(°C)             0
최저기온(°C)             0
합계 일조 시간(hr)       249
1시간 최다강수량(mm)    30912
일강수량(mm)         25111
최대 풍속(m/s)          38
지점_nm                0
년월                   0
월                    0
도                    0
dtype: int64
(41299, 13)


In [6]:
# 평균기온 지점 월 평균으로 대체
df.loc[df['평균기온(°C)'].isna(), '평균기온(°C)'] = df[df['평균기온(°C)'].isna()].apply(lambda x: df[(df['월']==x['월']) & (df['지점']==x['지점'])]['평균기온(°C)'].mean(), axis=1)

In [7]:
# 합계 일조 시간(hr) 지점 월 평균으로 대체
df.loc[df['합계 일조 시간(hr)'].isna(), '합계 일조 시간(hr)'] = df[df['합계 일조 시간(hr)'].isna()].apply(lambda x: df[(df['월']==x['월']) & (df['지점']==x['지점'])]['합계 일조 시간(hr)'].mean(), axis=1)

In [8]:
# 최대 풍속(m/s) 지점 월 평균으로 대체
df.loc[df['최대 풍속(m/s)'].isna(), '최대 풍속(m/s)'] = df[df['최대 풍속(m/s)'].isna()].apply(lambda x: df[(df['월']==x['월']) & (df['지점']==x['지점'])]['최대 풍속(m/s)'].mean(), axis=1)

In [9]:
# 나머지 결측치 0으로 채우기
df = df.fillna(value=0)

### 기온구간
- -5도 이하 > 0
- 5도 이하 > 1
- 15도 이하 > 2
- 25도 이하 > 3
- 35도 이하 > 4

In [10]:
df['기온구간'] = df['평균기온(°C)'].apply(lambda x : 0 if x< -5 
                                              else (1 if x<5 
                                                    else (2 if x<15 
                                                          else(3 if x<25 else 4
                                                              )
                                                         )
                                                   )
                                             )
print(df['기온구간'].value_counts())

3    14893
2    11862
1    10161
4     3059
0     1324
Name: 기온구간, dtype: int64


### 일조시간구간
- 3시간 이하 >> 0
- 6시간 이하 >> 1
- 9시간 이하 >> 2
- 12시간 이하 >> 3
- 15시간 이하 >> 4

In [11]:
df['일조시간구간'] = df['합계 일조 시간(hr)'].apply(lambda x : 0 if x< 3 
                                              else (1 if x<6 
                                                    else (2 if x<9 
                                                          else(3 if x<12 else 4
                                                              )
                                                         )
                                                   )
                                             )
df['일조시간구간'].value_counts()

2    12134
0    11345
3    10124
1     6478
4     1218
Name: 일조시간구간, dtype: int64

### 최고기온

In [12]:
df['최고기온일수'] = df['최고기온(°C)'].apply(lambda x : 1 if x >=30 else 0)
df['최고기온일수'].value_counts()

0    38326
1     2973
Name: 최고기온일수, dtype: int64

### 최저기온일수
- 배추가 -8도이하 일 경우, 동해를 입으므로 -8이하를 최저기온으로 정함

In [13]:
df['최저기온일수'] = df['최저기온(°C)'].apply(lambda x : 1 if x<=-8 else 0)
df['최저기온일수'].value_counts()

0    38973
1     2326
Name: 최저기온일수, dtype: int64

### 1시간 최다강수량기간
- 10mm미만 >> 0
- 20mm미만 >> 1
- 40mm미만 >> 2
- 40mm이상 >> 3

In [14]:
df['1H최다강수량'] = df['1시간 최다강수량(mm)'].apply(lambda x : 0 if x< 10 
                                              else (1 if x<20
                                                    else (2 if x<40 else 3)
                                                    )
                                                )
df['1H최다강수량'].value_counts()

0    40047
1      854
2      343
3       55
Name: 1H최다강수량, dtype: int64

### 강수량
- 0mm > 0
- 40mm이하 > 1
- 80mm이하 > 2
- 110mm이하 > 3
- 110mm이상 > 4

In [15]:
df['강수량구간'] = df['일강수량(mm)'].apply(lambda x : 0 if x== 0 
                                              else (1 if x<40
                                                    else (2 if x<80 
                                                          else(3 if x<110 else 4
                                                              )
                                                         )
                                                   )
                                             )
df['강수량구간'].value_counts()

0    28360
1    12019
2      660
3      139
4      121
Name: 강수량구간, dtype: int64

### 최대풍속
- 4 미만 > 0
- 10 미만 > 1
- 14 미만 > 2
- 14 이상 > 3

In [16]:
df['최대풍속'] = df['최대 풍속(m/s)'].apply(lambda x : 0 if x< 4
                                              else (1 if x<10
                                                    else (2 if x<14 else 3)
                                                    )
                                                )
df['최대풍속'].value_counts()

1    30083
0    10073
2     1059
3       84
Name: 최대풍속, dtype: int64

## 월단위 변환

In [17]:
df_month = df[['년월','도','기온구간','일조시간구간','최고기온일수','최저기온일수','1H최다강수량','강수량구간','최대풍속']].copy()
df_month.head()

,년월,도,기온구간,일조시간구간,최고기온일수,최저기온일수,1H최다강수량,강수량구간,최대풍속
0,1980-01,강원도,1,0,0,0,0,1,0
1,1980-01,강원도,2,0,0,0,0,1,0
2,1980-01,강원도,2,1,0,0,0,0,1
3,1980-01,강원도,1,0,0,0,0,1,1
4,1980-01,강원도,1,2,0,0,0,0,1


In [18]:
# 각각 one-hot-encoding설정
tem_ohe = pd.get_dummies(df_month['기온구간'])
tem_ohe.columns = ['평균기온(-5미만)','평균기온(-5~5미만)','평균기온(5~15미만)','평균기온(15~25미만)','평균기온(25~35미만)']

sun_ohe = pd.get_dummies(df_month['일조시간구간'])
sun_ohe.columns = ['일조시간(3미만)','일조시간(3~6미만)','일조시간(6~9미만)','일조시간(9~12미만)','일조시간(12~15미만)']

rain_1h_ohe = pd.get_dummies(df_month['1H최다강수량'])
rain_1h_ohe.columns = ['1H최다강수량(10미만)','1H최다강수량(10~20미만)','1H최다강수량(20~40미만)','1H최다강수량(40이상)']

rain_ohe = pd.get_dummies(df_month['강수량구간'])
rain_ohe.columns = ['일강수량(0)','일강수량(1~40미만)','일강수량(40~80미만)','일강수량(80~110미만)','일강수량(110이상)']

wind_ohe = pd.get_dummies(df_month['최대풍속'])
wind_ohe.columns = ['최대풍속(0~4미만)','최대풍속(4~10미만)','최대풍속(10~14미만)','최대풍속(14이상)']


df_month = pd.concat([df_month,tem_ohe,sun_ohe,rain_1h_ohe,rain_ohe,wind_ohe],axis=1)

In [19]:
df = pd.DataFrame(df_month.groupby(['년월','도'])[
    '평균기온(-5미만)','평균기온(-5~5미만)','평균기온(5~15미만)','평균기온(15~25미만)','평균기온(25~35미만)',
    '일조시간(3미만)','일조시간(3~6미만)','일조시간(6~9미만)','일조시간(9~12미만)','일조시간(12~15미만)',
    '최고기온일수','최저기온일수',
    '1H최다강수량(10미만)','1H최다강수량(10~20미만)','1H최다강수량(20~40미만)','1H최다강수량(40이상)',
    '일강수량(0)','일강수량(1~40미만)','일강수량(40~80미만)','일강수량(80~110미만)','일강수량(110이상)',
    '최대풍속(0~4미만)','최대풍속(4~10미만)','최대풍속(10~14미만)','최대풍속(14이상)'
].sum())
df.shape

(950, 25)

In [20]:
df.reset_index(level=['도'], inplace=True)

In [21]:
df.columns

Index(['도', '평균기온(-5미만)', '평균기온(-5~5미만)', '평균기온(5~15미만)', '평균기온(15~25미만)',
       '평균기온(25~35미만)', '일조시간(3미만)', '일조시간(3~6미만)', '일조시간(6~9미만)',
       '일조시간(9~12미만)', '일조시간(12~15미만)', '최고기온일수', '최저기온일수', '1H최다강수량(10미만)',
       '1H최다강수량(10~20미만)', '1H최다강수량(20~40미만)', '1H최다강수량(40이상)', '일강수량(0)',
       '일강수량(1~40미만)', '일강수량(40~80미만)', '일강수량(80~110미만)', '일강수량(110이상)',
       '최대풍속(0~4미만)', '최대풍속(4~10미만)', '최대풍속(10~14미만)', '최대풍속(14이상)'],
      dtype='object')

# 배추가격 전처리

In [22]:
df2 = pd.read_csv('1997kamis.csv')

In [23]:
df2['년월'] = df2['date'].apply(lambda x: str(x)[0:4] + '-' + str(x)[4:6])

In [24]:
df2 = df2[['price','년월']].groupby(['년월']).mean()

In [25]:
final_df = pd.merge(df, df2, on='년월', how='left')

In [26]:
# 시계열 데이터는 같은 날짜의 컬럼을 두개 가져갈수 없으므로 수정
temp_df1 = final_df[final_df['도'] == '강원도'][[
    '평균기온(-5미만)','평균기온(-5~5미만)','평균기온(5~15미만)','평균기온(15~25미만)','평균기온(25~35미만)',
    '일조시간(3미만)','일조시간(3~6미만)','일조시간(6~9미만)','일조시간(9~12미만)','일조시간(12~15미만)',
    '최고기온일수','최저기온일수',
    '1H최다강수량(10미만)','1H최다강수량(10~20미만)','1H최다강수량(20~40미만)','1H최다강수량(40이상)',
    '일강수량(0)','일강수량(1~40미만)','일강수량(40~80미만)','일강수량(80~110미만)','일강수량(110이상)',
    '최대풍속(0~4미만)','최대풍속(4~10미만)','최대풍속(10~14미만)','최대풍속(14이상)'
]]

In [27]:
temp_df2 = final_df[final_df['도'] == '전라도'][[
    '평균기온(-5미만)','평균기온(-5~5미만)','평균기온(5~15미만)','평균기온(15~25미만)','평균기온(25~35미만)',
    '일조시간(3미만)','일조시간(3~6미만)','일조시간(6~9미만)','일조시간(9~12미만)','일조시간(12~15미만)',
    '최고기온일수','최저기온일수',
    '1H최다강수량(10미만)','1H최다강수량(10~20미만)','1H최다강수량(20~40미만)','1H최다강수량(40이상)',
    '일강수량(0)','일강수량(1~40미만)','일강수량(40~80미만)','일강수량(80~110미만)','일강수량(110이상)',
    '최대풍속(0~4미만)','최대풍속(4~10미만)','최대풍속(10~14미만)','최대풍속(14이상)','price'
]]

In [28]:
temp_df1.columns = [
    'J평균기온(-5미만)','J평균기온(-5~5미만)','J평균기온(5~15미만)','J평균기온(15~25미만)','J평균기온(25~35미만)',
    'J일조시간(3미만)','J일조시간(3~6미만)','J일조시간(6~9미만)','J일조시간(9~12미만)','J일조시간(12~15미만)',
    'J최고기온일수','J최저기온일수',
    'J1H최다강수량(10미만)','J1H최다강수량(10~20미만)','J1H최다강수량(20~40미만)','J1H최다강수량(40이상)',
    'J일강수량(0)','J일강수량(1~40미만)','J일강수량(40~80미만)','J일강수량(80~110미만)','J일강수량(110이상)',
    'J최대풍속(0~4미만)','J최대풍속(4~10미만)','J최대풍속(10~14미만)','J최대풍속(14이상)'
]

In [29]:
temp_df2.columns = [
    'G평균기온(-5미만)','G평균기온(-5~5미만)','G평균기온(5~15미만)','G평균기온(15~25미만)','G평균기온(25~35미만)',
    'G일조시간(3미만)','G일조시간(3~6미만)','G일조시간(6~9미만)','G일조시간(9~12미만)','G일조시간(12~15미만)',
    'G최고기온일수','G최저기온일수',
    'G1H최다강수량(10미만)','G1H최다강수량(10~20미만)','G1H최다강수량(20~40미만)','G1H최다강수량(40이상)',
    'G일강수량(0)','G일강수량(1~40미만)','G일강수량(40~80미만)','G일강수량(80~110미만)','G일강수량(110이상)',
    'G최대풍속(0~4미만)','G최대풍속(4~10미만)','G최대풍속(10~14미만)','G최대풍속(14이상)','price'
]

In [31]:
final_df = pd.concat([temp_df1, temp_df2], axis=1)

In [32]:
final_df.to_csv('cabbage_data.csv', encoding='utf-8-sig')

In [35]:
final_df.shape

(475, 51)